<a href="https://colab.research.google.com/github/GaoangLiu/AA_ipynb/blob/master/Sentiment_Analysis_on_Movie_Reviews_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Download data 
!rm *
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece
!wget -O movie.zip ali.140714.xyz:8000/sentiment_analysis.zip 
!wget -O b7.py ali.140714.xyz:8000/boost117.py
!unzip movie.zip 
!ls

rm: cannot remove '__pycache__': Is a directory
rm: cannot remove 'sample_data': Is a directory
--2020-05-21 08:28:31--  http://ali.140714.xyz:8000/sentiment_analysis.zip
Resolving ali.140714.xyz (ali.140714.xyz)... 47.240.16.188
Connecting to ali.140714.xyz (ali.140714.xyz)|47.240.16.188|:8000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1976135 (1.9M) [application/zip]
Saving to: ‘movie.zip’

movie.zip           100%[===================>]   1.88M  1.62MB/s    in 1.2s    

2020-05-21 08:28:33 (1.62 MB/s) - ‘movie.zip’ saved [1976135/1976135]

--2020-05-21 08:28:34--  http://ali.140714.xyz:8000/boost117.py
Resolving ali.140714.xyz (ali.140714.xyz)... 47.240.16.188
Connecting to ali.140714.xyz (ali.140714.xyz)|47.240.16.188|:8000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1723 (1.7K) [text/plain]
Saving to: ‘b7.py’

b7.py               100%[===================>]   1.68K  --.-KB/s    in 0s      

2020-05-21 08:28:34 (329 MB/s) - ‘b7.py’ s

In [0]:
# !pip install sentencepiece
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import logging

logging.basicConfig(level=logging.INFO)

In [0]:
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.tsv', sep='\t')

In [5]:
%%time
import tensorflow_hub as hub 
import tokenization
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

INFO:absl:Using /tmp/tfhub_modules to cache modules.


CPU times: user 4.31 s, sys: 769 ms, total: 5.08 s
Wall time: 12.8 s


In [0]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(5, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
max_len = 120
train_input = bert_encode(train.Phrase.values, tokenizer, max_len=max_len)
test_input = bert_encode(test.Phrase.values, tokenizer, max_len=max_len)
train_labels = tf.keras.utils.to_categorical(train.Sentiment.values, num_classes=5)

In [8]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 120)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 120)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 120)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [0]:
%%time
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    train_input, train_labels, 
    validation_split=0.2,
    epochs=1,
    callbacks=[checkpoint, earlystopping],
    batch_size=32,
    verbose=1
)

 395/3902 [==>...........................] - ETA: 4:54:31 - loss: 1.1488 - accuracy: 0.5396

## Make predictions

In [0]:
%%time 
model.load_weights('model.h5')
test_pred = model.predict(test_input)
# sub = pd.read_csv('sampleSubmission.csv')
# sub['Sentiment'] = test_pred.round().astype(int)
# sub.to_csv('tweets.csv', index=False)